<b> Lab 2 | Week 2 Day 2 </b> 

### First Agentic Framework Project!!

Build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools - to call functions and use agent-as-tool
3. Agent collaboration via Tools and Handoffs

Sendgrid signed up

Recovery Code: HUF5S11VARJR4KDZTLC77PPM

In [ ]:
import os, sendgrid, asyncio, openai
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict

In [ ]:
load_dotenv(override=True)

In [ ]:
# Let's the check send eamil function is working

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))

    from_email = Email("skillshift.ai@gmail.com") # SendGrid verified sender
    to_email = To("melbith@gmail.com")

    subject = "Test Email"
    content = Content("text/plain", "This is a test email using SendGrid")

    # mail = Mail(from_email, to_email, subject, content).get()
    # response = sg.client.mail.send.post(request_body=mail)

    mail = Mail(from_email, to_email, subject, content)
    response = sg.client.mail.send.post(request_body=mail.get())

    print(response.status_code)

In [ ]:
send_test_email()

___
<b> Stage 1. Agent workflow </b> 

In [ ]:
# Let's define the System Prompt/Instructions for the 3 sales agents

# Instructions for the professional agent
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

# Instructions for the engaging agent
instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

# Instructions for the busy agent
instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [ ]:
# Define the sales agents

# Professional Sales Agent
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

# Engaging Sales Agent
sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:
# Let's run the professional sales agent in streamed mode

result = Runner.run_streamed(
    sales_agent1,
    input="Write a cold sales email"
)
# print(result)
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)


In [ ]:
# Now let's run all 3 agents in parallel

message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Now Let's introduce a fourth agent and instruct it to pick the best email of the three

In [ ]:
# Introduce a fourthth agent that will pick the best email from the 3 agents

# Instructions for the best email picker agent
instructions4 = "You pick the best cold sales email from the given options. \
    Imagine you are a customer and pick the one you are most likely to respond to. \
    Do not give an explanation; reply with the selected email only."

# Define the best email picker agent
best_email_picker = Agent(
    name="Best Email Picker",
    instructions=instructions4,
    model="gpt-4o-mini"
)

In [ ]:
# Run the three agents in parallel and let the fourth one pick the best

message = "Write a cold sales email"

with trace("Selecting best sales email"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

    email_outputs = [result.final_output for result in results]

    emails = "Cold Sales Emails:\n\n" + "\n\nEmail:\n\n".join(email_outputs)

    # print(emails)

    best_email = await Runner.run(best_email_picker, emails)

    print(f"Best sales email:\n{best_email.final_output}")

Now go and check out the trace: https://platform.openai.com/traces

___
<b> Stage 2. Use of tools - to call functions and use agent-as-tool</b> 

This Orchestration will have a top level Planning Agent (Sales Manager) and 3 Sales Agents (that generate cold emails) 

The idea here is to facilitate the use tools for the Planning Agent. It will have access to the 3 sales agents as tools, NOT AS AGENTS,
to generate Cold Emails. It has to then pick the single best email from the list and send it using the 4th tool
which is a function tool

In [ ]:
# Define the sales agents

# Professional Sales Agent
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

# Engaging Sales Agent
sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

# Busy Sales Agent
sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:
sales_agent2

Define the send email function and simply wrap it with the decorator `@function_tool` so that it becomes a tool

In [ ]:
@function_tool
def send_email(subject: str, body: str):
    """ Send out an email with the given subject and body to all sales prospects """

    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))

    from_email = Email("skillshift.ai@gmail.com") # SendGrid verified sender
    to_email = To("melbith@gmail.com")

    # subject = "Sales Email"
    content = Content("text/plain", body)

    mail = Mail(from_email, to_email, subject, content)
    sg.client.mail.send.post(request_body=mail.get())

    return{"status":"success"}

In [ ]:
# Let's verify it
send_email

Agents can also be converted as tool using the .as_tool() construct

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

#### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents, 
and a tool for our function to send emails

In [ ]:
tools = [tool1, tool2, tool3, send_email]

# tools

In [ ]:
# Here is cleaner way to gather the tools, using loop and list comprehension(enumerator)

description = "Write a cold sales email"

agents = [sales_agent1, sales_agent2, sales_agent3]

tools = [
    agent.as_tool(
        tool_name=f"sales_agent{i+1}",
        tool_description=description
    )
    for i, agent in enumerate(agents)
]

tools.append(send_email)

In [ ]:
tools

And now it's time to introduce our Planning Agent (Sales Manager)

In [ ]:
# Instructions for the Planning Agent

instructions = """
You are a Sales Manager working for ComplAI. Your goal is to generate cold sales emails using the sales_agent tools, pick the best
email using your judgement and send it to the sales prospects using the send_email tool

## Tools

- You have access to three sales_agent tools to generate cold sales emails. Use these tools given to generate cold sales emails. 
  Do not do it yourself; always use these tools
- Use your judgment to choose the best email
- You have access to the send_email tool. Use this tool to send the best email (and only the best email) to the user

## Important Rules

- You must use the sales_agent tools to generate the drafts — do not write them yourself.
- After picking the best email, pass the subject and body seperately to the send_email tool. 
- Make sure that the Subject line is removed from the mail body and it always begins with 'Dear CEO'.
- You must send only ONE email using the send_email tool — never more than one.
"""

In [ ]:
# Define the Planning Agent

sales_manager = Agent(
    name="Sales Manager", 
    instructions=instructions, 
    tools=tools, 
    model="gpt-4o-mini"
    )

# Execute

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales Manager"):
    result = await Runner.run(sales_manager, message)

Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

___

<b> Stage 3. Agent Collaboration via Handoffs & Tools </b> 

<i>Handoffs represent a way an agent can delegate to an agent, passing control to it</i>

Handoffs and Agents-as-tools are similar:

        In both cases, an Agent can collaborate with another Agent
        With tools, control passes back
        With handoffs, control passes across

Multi-agent system for generating and sending cold sales emails. Uses OpenAI agents with a hierarchical structure: three sales agents generate drafts, a sales manager selects the best, and an email manager formats and sends it.

Functionality Flow:

- Three sales agents generate email drafts with different styles
- Sales manager selects the best email
- Email manager formats (subject + HTML conversion) and sends the email
- All steps are traced for debugging

This orchestration will have:

A top level Planning Agent (Sales Manager) that has access to:

        Three sales agents (that generate cold emails) as tools
        Emailer agent as Handoff

The Handoff, i.e. the Emailer agent, has access to:

        A subject writer and an html converter agents as tools
        A send_html_email function tool

The idea here is to facilitate the use tools and Handoffs for the Planning Agent.

In [ ]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-5-mini")
subject_writer_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-5-mini")
html_converter_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [ ]:

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """

    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))

    from_email = Email("skillshift.ai@gmail.com") # SendGrid verified sender
    to_email = To("melbith@gmail.com")

    content = Content("text/html", html_body)

    mail = Mail(from_email, to_email, subject, content)
    # sg.client.mail.send.post(request_body=mail.get())

    try:
        sg.client.mail.send.post(request_body=mail.get())
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

    # return{"status":"success"}


In [ ]:
emailer_tools = [subject_writer_tool, html_converter_tool, send_html_email]

In [ ]:
emailer_tools

In [ ]:

emailer_instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=emailer_instructions,
    tools=emailer_tools,
    model="gpt-5-mini",
    handoff_description="Convert an email to HTML and send it")


In [ ]:
# Define the tools and handoffs for the top level Sales Manager

sales_manager_tools = [tool1, tool2, tool3] # 3 sales agents as tools
sales_manager_handoffs = [emailer_agent]
print(sales_manager_tools)
print(sales_manager_handoffs)

In [ ]:
# Instructions for the Planning Agent

# sales_manager_instructions = """
# You are a Sales Manager working for ComplAI. Your goal is to use the three sales_agent tools to generate cold sales emails, 
# pick the best email using your judgement and send it to the sales prospects using the 'Email Manager' agent

# ## Tools

# - You have access to three sales_agent tools to generate cold sales emails. Use these tools to generate cold sales emails. 
#   Do not do it yourself; always use these tools
# - Use your judgment to choose the best email. You can use the tools multiple times if you're not satisfied with the results from the first try.

# ## Handoffs

# - You have access to the 'Email Manager' agent. Pass ONLY the winning email to the 'Email Manager' agent. 
#   The Email Manager will take care of formatting and sending.

# ## Important Rules

# - You must use the sales_agent tools to generate the drafts — do not write them yourself.
# - After picking the best email, pass it to the 'Email Manager' agent. 
# - You must hand off exactly ONE email to the Email Manager — never more than one.
# """

sales_manager_instructions = """
You are the Sales Manager for ComplAI. Your responsibility is to generate and select the most effective cold sales email using the tools and 
agents available to you.

## Your Objective
Use the three sales_agent tools to generate multiple cold sales email drafts. Compare them and select the single best email — the one 
most likely to get a positive response from the prospect. Then hand off ONLY that winning email to the 'Email Manager' agent for final formatting and sending.

## Tools — Mandatory Usage
- You have access to three sales_agent tools. Each tool has a distinct persona and produces different styles of cold sales emails.
- You MUST call all three sales_agent tools at least once in every task.
- If you are not satisfied with a draft from a tool, you MAY call that tool again to request another version.
- You absolutely MUST NOT write or edit any cold sales emails yourself. All drafts must come from the sales_agent tools.

## Choosing the Best Email
Use your judgment to select the single best email. The “best” email is defined as:
- the email most likely to get a reply from the prospect,
- the most persuasive, clear, and compelling message,
- the one with the strongest call to action and highest likelihood of engagement.

You may compare drafts, analyze strengths and weaknesses, and request additional versions before deciding.

## Handoffs
- After selecting the winning email, you MUST hand off exactly ONE email to the 'Email Manager' agent.
- Do not send more than one email.
- Do not format or modify the content yourself — the Email Manager will take care of that.

## Important Rules
- Do NOT generate any email content yourself.
- Do NOT skip any of the three tools; each must be used at least once.
- Do NOT hand off more than one email.
- Your final action must be a handoff to the 'Email Manager' agent containing only the winning email.

Follow these instructions strictly to achieve the best outcome.
"""   

In [ ]:
sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=sales_manager_tools,
    handoffs=sales_manager_handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

Go check the trace @
https://platform.openai.com/traces

And then check your email!!